In [ ]:
! pip install streamlit torch torchaudio pydub transformers langchain_google_genai langchain_core langchain_community chromadb sentence-transformers scikit-learn numpy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%writefile app2.py
import streamlit as st
import torch
import torchaudio
import os
import tempfile
import numpy as np
from pydub import AudioSegment
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableLambda
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate

# Load Whisper model from Hugging Face
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

# Load Gemini API Key
with open("api.txt", "r") as file:
    GOOGLE_API_KEY = file.read().strip()

# Initialize Gemini AI model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY)

# Initialize ChromaDB
persist_directory = "/content/drive/MyDrive/files/chroma_db"
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = Chroma(collection_name="vector_database", embedding_function=embeddings_model, persist_directory=persist_directory)

# Retriever
retriever = db.as_retriever(search_kwargs={"k": 3})

# Prompt Template
prompt_template = PromptTemplate(
    input_variables=["chat_history", "series_name", "reasoning", "question"],
    template=(
        "You are an AI assistant that provides direct answers based ONLY on the given context.\n\n"
        "Previous Conversation History:\n{chat_history}\n\n"
        "Series Name: {series_name}\n\n"
        "Relevant Events Leading to the Answer:\n{reasoning}\n\n"
        "User Question:\n{question}\n"
        "Final Answer (No extra text, just the answer):"
    )
)

# Chat History Storage
chat_history = []

# Function: Retrieve & Process Data
def retrieve_and_process(inputs):
    query = inputs["question"]
    retrieved_docs = retriever.invoke(query)

    if not retrieved_docs:
        return {
            "chat_history": "",
            "series_name": "Unknown",
            "reasoning": "No relevant events found.",
            "question": query
        }

    extracted_texts = [doc.page_content for doc in retrieved_docs]
    series_names = [doc.metadata.get("series_name", "Unknown") for doc in retrieved_docs]

    # 🔹 Generate Reasoning
    reasoning_prompt = f"Summarize the most relevant events from '{series_names[0]}' before answering: {query}.\n\nContext:\n" + "\n".join(extracted_texts)
    reasoning = llm.invoke(reasoning_prompt).content.strip()

    formatted_chat_history = "\n".join([f"User: {entry['user']}\nAssistant: {entry['assistant']}" for entry in chat_history])

    return {
        "chat_history": formatted_chat_history,
        "series_name": series_names[0],
        "reasoning": reasoning,
        "question": query
    }

# RAG Chain
  retrieval_chain = (
      RunnableLambda(retrieve_and_process)
      | (lambda x: {**x, "answer": llm.invoke(prompt_template.format(**x)).content.strip()})  # Ensure dictionary output
  )

# Function: Process Audio & Transcribe using Hugging Face Whisper
def process_audio(audio_file):
    temp_audio_path = tempfile.NamedTemporaryFile(delete=False, suffix=".wav").name

    # Convert to 16kHz WAV
    audio = AudioSegment.from_file(audio_file)
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export(temp_audio_path, format="wav")

    # Load audio
    speech, sr = torchaudio.load(temp_audio_path)
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
    speech = resampler(speech).squeeze()

    # Process the input audio
    input_features = processor(speech, sampling_rate=16000, return_tensors="pt").input_features

    # Generate transcription
    with torch.no_grad():
        predicted_ids = whisper_model.generate(input_features)

    # Decode and return transcription
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    # Remove temp file
    os.remove(temp_audio_path)

    return transcription

# Function: Ask Question (Retrieve & Generate Response)
def ask_question(user_query):
    global chat_history

    result = retrieval_chain.invoke({"question": user_query})

    # Result is a dictionary
    if not isinstance(result, dict):
        raise TypeError(f"Unexpected output from retrieval chain: {result}")

    # Extract fields safely
    answer = result.get("answer", "No answer found.")
    series_name = result.get("series_name", "Unknown")
    reasoning = result.get("reasoning", "No reasoning found.")

    # Store in Chat History
    chat_history.append({
        "user": user_query,
        "assistant": answer,
        "series_name": series_name,
        "reasoning": reasoning
    })

    return series_name, reasoning, answer

# Streamlit UI
st.title("🔍 AI-Powered Video Subtitle Search")

# Upload Audio File
uploaded_file = st.file_uploader("📂 Upload an audio file", type=["mp3", "wav", "w4a", "m4a", "ogg", "flac"])

# Manual Text Input
user_input = st.text_area("✍️ Or manually enter text")

if uploaded_file:
    st.audio(uploaded_file, format="audio/wav")
    if st.button("🎤 Transcribe & Search"):
        with st.spinner("Processing audio..."):
            transcribed_text = process_audio(uploaded_file)
        st.subheader("📝 Transcription")
        st.write(transcribed_text)
        with st.spinner("🔎 Searching for relevant subtitles..."):
            series_name, reasoning, answer = ask_question(transcribed_text)
        st.subheader("📌 Search Results")
        st.write(f"**🎬 Series Name:** {series_name}")
        st.write(f"**📜 Relevant Events:** {reasoning}")
        st.write(f"**✅ Answer:** {answer}")

if st.button("🔍 Submit"):
    if user_input.strip():
        with st.spinner("🔎 Searching for relevant subtitles..."):
            series_name, reasoning, answer = ask_question(user_input)
        st.subheader("📌 Search Results")
        st.write(f"**🎬 Series Name:** {series_name}")
        st.write(f"**📜 Relevant Events:** {reasoning}")
        st.write(f"**✅ Answer:** {answer}")
    else:
        st.warning("⚠️ Please enter some text before submitting.")
